In [ ]:
##
#Load Packages
import numpy as np
import matplotlib.pyplot as plt
from numba import jit
import sys
import nibabel as nib
import time

In [ ]:
##
#Define Path to Code database
DirPath = '/Users/btendler/Documents/MotionCorrection/Paper/Code/'

#Define Output Path
OutputPath = '/Users/btendler/Documents/MotionCorrection/Paper/SupportingInformation/Datasets/'

In [ ]:
##
#Load functions
sys.path.append(''.join([DirPath,'bin']))
from EPGMotion import *
from ParameterOptionsSimulation import *
from MotionSimulation import *

In [ ]:
##
#Read Parameters
opt=ParameterOptionsSimulation()

In [ ]:
##
#Conventional EPG simulates a maximum k-value pathway equal to the number of TRs. This piece of code identifies where increasing the k-value leads to no meaningful change in the signal, with subsequent thresholding to accelerate modelling
opt["kOrder"] = kOrder(opt)

In [ ]:
##
#Load diffusion coefficient map (HCP1065 MD map) & Affine
D = np.atleast_3d(nib.load(''.join(['/usr/local/fsl/data/standard/FSL_HCP1065_MD_1mm.nii.gz'])).get_fdata())
Affine = nib.load(''.join(['/usr/local/fsl/data/standard/FSL_HCP1065_MD_1mm.nii.gz'])).affine

##
#Get Voxel Dimensions (in mm)
VoxDims = nib.affines.voxel_sizes(Affine)

##
#Create Brain Mask 
Mask = D > 0

##
#Obtain mask indices
MaskIdx = np.where(Mask==1)

##
#Vectorise Diffusion & Mask map for dictionary to accelerate EPG fitting
opt['D'] = np.asarray([*D[Mask==1]], dtype='f8')
opt['Mask'] = np.asarray([*Mask[Mask==1]], dtype='f8')

In [ ]:
##

##
#Define T1 and T2 values as equal across the simulation
opt['T1'] = opt['Mask']*opt['T1']
opt['T2'] = opt['Mask']*opt['T2']

##
#Define B1 as constant across the image (i.e. no B1 inhomogeneity)
opt['B1'] = np.asarray([*Mask[Mask==1]], dtype='f8') 

In [ ]:
##
#Define Motion Operators for each simulation - Translation Operator (mm/s), Rotation Operator (rad/s) & Cardiac Operator (vector defines maximum beat velocity in mm/s)

##
#Define consistent motion per TR 
RandMotion=False
#Define Heart Rate for Cardiac Operator (beats/min)
HeartRate = 50

##
#Define Operators (Translation Example)
TranslationExample_Trans= np.array([0,0,0.2])
TranslationExample_Rot = np.array([0,0,0])
TranslationExample_Card = np.array([0,0,0])

##
#Define Operators (Rotation Example)
RotationExample_Trans= np.array([0,0,0])
RotationExample_Rot = np.array([0.2/180*np.pi,0,0])
RotationExample_Card = np.array([0,0,0])

##
#Define Operators (Cardiac Example)
CardiacExample_Trans= np.array([0,0,0])
CardiacExample_Rot = np.array([0,0,0])
CardiacExample_Card = np.array([0,0,1.5])


In [ ]:
##
#Simulation - No Motion

##
#Initialise Output Array
F = np.zeros((*Mask.shape,opt['nTR'].astype(np.int32)[0]), dtype='c8')

##
#Perform Forward Simulation
start = time.time()
F[MaskIdx[0],MaskIdx[1],MaskIdx[2],:] = EPGMotionWholeImage(opt)
end = time.time()
print(end - start)


##
#Save Data
nib.save(nib.Nifti1Image(np.abs(F)*Mask[:,:,:,np.newaxis],Affine),''.join([OutputPath, '/NoMotion/Magnitude.nii.gz']))
nib.save(nib.Nifti1Image(np.angle(F*-1)*Mask[:,:,:,np.newaxis],Affine),''.join([OutputPath, '/NoMotion/Phase.nii.gz']))
nib.save(nib.Nifti1Image(F*0,Affine),''.join([OutputPath, '/NoMotion/T.nii.gz']))

##
#Clear arrays
F = []

In [ ]:
##
#Simulation - Translational Motion

##
#Initialise Output Array
FTrans = np.zeros((*Mask.shape,opt['nTR'].astype(np.int32)[0]), dtype='c8')

##
#Establish Velocity Timeseries Profile
start = time.time()
T_Trans = MotionOperator(opt,TranslationExample_Trans,TranslationExample_Rot,TranslationExample_Card,Mask,VoxDims)

##
#Peform Forward Simulation
FTrans[MaskIdx[0],MaskIdx[1],MaskIdx[2],:] = EPGMotionWholeImage(opt,T_Trans[Mask==1,:])
end = time.time()
print(end - start)


##
#Save Data
nib.save(nib.Nifti1Image(np.abs(FTrans)*Mask[:,:,:,np.newaxis],Affine),''.join([OutputPath, '/Translation/Magnitude.nii.gz']))
nib.save(nib.Nifti1Image(np.angle(FTrans*-1)*Mask[:,:,:,np.newaxis],Affine),''.join([OutputPath, '/Translation/Phase.nii.gz']))
nib.save(nib.Nifti1Image(T_Trans*Mask[:,:,:,np.newaxis],Affine),''.join([OutputPath, '/Translation/T.nii.gz']))

##
#Clear arrays
T_Trans = []
FTrans = []


In [ ]:
##
#Simulation - Rotational Motion

##
#Initialise Output Array
FRot = np.zeros((*Mask.shape,opt['nTR'].astype(np.int32)[0]), dtype='c8')

##
#Establish Velocity Timeseries Profile
start = time.time()
T_Rot = MotionOperator(opt,RotationExample_Trans,RotationExample_Rot,RotationExample_Card,Mask,VoxDims)

##
#Peform Forward Simulation
FRot[MaskIdx[0],MaskIdx[1],MaskIdx[2],:] = EPGMotionWholeImage(opt,T_Rot[Mask==1,:])
end = time.time()
print(end - start)

##
#Save Data
nib.save(nib.Nifti1Image(np.abs(FRot)*Mask[:,:,:,np.newaxis],Affine),''.join([OutputPath, '/Rotation/Magnitude.nii.gz']))
nib.save(nib.Nifti1Image(np.angle(FRot*-1)*Mask[:,:,:,np.newaxis],Affine),''.join([OutputPath, '/Rotation/Phase.nii.gz']))
nib.save(nib.Nifti1Image(T_Rot*Mask[:,:,:,np.newaxis],Affine),''.join([OutputPath, '/Rotation/T.nii.gz']))

##
#Clear arrays
T_Rot = []
FRot = []

In [ ]:
##
#Simulation - Cardiac Motion

##
#Initialise Output Array
FCardiac = np.zeros((*Mask.shape,opt['nTR'].astype(np.int32)[0]), dtype='c8')

##
#Establish Velocity Timeseries Profile
start = time.time()
T_Cardiac = MotionOperator(opt,CardiacExample_Trans,CardiacExample_Rot,CardiacExample_Card,Mask,VoxDims,HeartRate=HeartRate)

##
#Peform Forward Simulation
FCardiac[MaskIdx[0],MaskIdx[1],MaskIdx[2],:] = EPGMotionWholeImage(opt,T_Cardiac[Mask==1,:])
end = time.time()
print(end - start)


##
#Save Data
nib.save(nib.Nifti1Image(np.abs(FCardiac)*Mask[:,:,:,np.newaxis],Affine),''.join([OutputPath, '/Cardiac/Magnitude.nii.gz']))
nib.save(nib.Nifti1Image(np.angle(FCardiac*-1)*Mask[:,:,:,np.newaxis],Affine),''.join([OutputPath, '/Cardiac/Phase.nii.gz']))
nib.save(nib.Nifti1Image(T_Cardiac*Mask[:,:,:,np.newaxis],Affine),''.join([OutputPath, '/Cardiac/T.nii.gz']))

##
#Clear arrays
T_Cardiac = []
FCardiac = []